# Table of contents
* [How to Run GUI Applications in a Docker Container](#how-to-run-gui-applications-in-a-docker-container)
    * [Forwarding An X Socket to A Docker Container](#forwarding_an_x_socket_to_a_cocker_container)
    * [Handling X Authentication](#handling-x-authentication)
    * [Docker Command](#docker_command)
    * [Forwarding the X server to a Docker container](#forwarding-the-x-server-to-a-docker-container)



# [How to Run GUI Applications in a Docker Container](https://www.howtogeek.com/devops/how-to-run-gui-applications-in-a-docker-container/)
Docker's normally used to containerise background applications and CLI programs. You can also use it to run graphical programs though! You can either use an existing X Server, where the host machine is already running a graphical environment, or you can run a VNC server within the container.

First it's important to understand what Docker actually does. A Docker "container" is a form of *encapsulation* which seems to be superficially similar to a virtual machine. Unlike a virtual machine, containers share the same Linux kernel as their host system.

The next component is the X Window System. <font color=green>**X Servers such as Xorg provide the fundamental graphical capabilities of Unix systems. GUI applications can't render without an X Server available**</font>. (Alternative windowing systems, such as Wayland, are available - we're focusing on X in this article.)

Trying to run an X Server in Docker is theoretically possible but rarely used. You'd need to run Docker in privileged mode
```shell
 --privileged
```
so it could access your host's hardware. Starting the server would try to claim your video devices, usually resulting in loss of video output as your host's original X server gets its devices yanked away.

<font color='green'>**A better approach is to mount your host's X Server socket into the Docker container**</font>. This allows your container to use the X Server you already have. GUI applications running in the container would then appear on your existing desktop.

## Forwarding An X Socket to A Docker Container

Providing a Docker container with access to your host's X socket is a straightforward procedure. The X socket can be found in
```shell
/tmp/.X11-unix
```
on your host. The contents of this directory should be mounted into a Docker volume assigned to the container. You'll need to use the `host` networking mode for this to work.

You must also provide the container with a `DISPLAY` environment variable. This instructs X clients - your graphical programs - which X server to connect to. Set `DISPLAY` in the container to the value of `$DISPLAY` on your host.

You can encapsulate all this configuration in one `docker-compose.yml` file:
```txt
version: "3"
services:
app:
    image: my-app:latest
    build: .
    environment:
    - DISPLAY=${DISPLAY}
    volumes:
    - /tmp/.X11-unix:/tmp/.X11-unix
    network_mode: host
```

This approach should only be used when you trust your Docker container. Exposing the host's display server is a security risk if you're not completely sure what lies inside the container.


## Handling X Authentication

You might need to authenticate the container to access the X Server. First get an X authentication token from your host machine. Run `xauth list` and note down one of the listed cookies. You'll need to copy the entire line.

![xauth_list](https://static1.howtogeekimages.com/wordpress/wp-content/uploads/csit/2021/04/4a47a0db.png?q=50&fit=crop&w=943&dpr=1.5)

Inside the Docker container, install the xauth package. Then run `xauth add`, passing the token you copied in the previous step.
```shell
apt install -y xauth
xauth add <token>
```
Your container should now successfully authenticate to the X Server.


## Docker Command
```shell
docker run --rm -it \
           -u $(id -u ${USER}):$(id -g ${USER}) \
           -v $(pwd)/flow:/OpenROAD-flow-scripts/flow \
           -e DISPLAY=${DISPLAY} \
           -v /tmp/.X11-unix:/tmp/.X11-unix \
           -v ${HOME}/.Xauthority:/.Xauthority \
           --network host \
           --security-opt seccomp=unconfined \
           IMAGE_NAME
```

## Forwarding the X server to a Docker container

Forwarding the X server to a Docker container allows GUI applications running inside the container to display their windows on the host's desktop. Here are the steps to forward the X server to a container:

1. Install Necessary Software on the Host:
Ensure that you have x11-xserver-utils installed on your host machine to use xhost.
```bash
    sudo apt-get install x11-xserver-utils
```

2. Allow Local X11 Connections:
Use the xhost command to allow local connections to your X server:
```bash
    xhost +local:
```
This allows any local user to connect to your X server. It's important to be cautious with this setting, especially in multi-user environments or on networks you don't trust.

3. Run the Docker Container:
When you run your Docker container, you'll need to set the DISPLAY environment variable and bind-mount the X11 Unix socket.
```bash
    docker run -it \
    -e DISPLAY=$DISPLAY \
    -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
    your-image
```
* -e DISPLAY=$DISPLAY: This sets the DISPLAY environment variable inside the container to the same value as on the host. This tells GUI applications inside the container where to send their display output.
* -v /tmp/.X11-unix:/tmp/.X11-unix:ro: This bind-mounts the X11 Unix socket from the host into the container. GUI applications use this socket to communicate with the X server.

4. Run Your GUI Application:
Inside the container, you should now be able to run your GUI application, and its window should appear on your host's desktop.

5. Revoke Access (Optional):
Once you're done, you can revoke the X server access you granted earlier using:
```bash
    xhost -local:
```
Note: Forwarding the X server like this can have security implications. Be sure to only run trusted applications inside the container and avoid using this method on untrusted networks. If you need more security, consider solutions like using xpra or VNC to create an isolated X environment for your container.